# Lightweight Fine-Tuning Project

Description of Choices

* PEFT technique: LoRA (Low-Rank Adaptation) is the selected PEFT techniques since it is compatible with all models
* Model: GPT-2 is the selected model since it is relatively small and compatible with sequence classification and LoRA
* Evaluation approach: Evaluation will be conducted using Hugging Face's `Trainer` class which simplifies the training and evaluation workflow. accuracy and F1-score are computed using `compute_metrics` function which allaows for a comparison of the performance of the original model against the fine-tunned model
* Fine-tuning dataset: The IMDb dataset from the Hugging Face `datasets` library will be used for fine-tunning. This dataset is a standard benchmark for binary sentiment classification tasks making it a good fit in this context.

## Loading and Evaluating Foundation Model

Load GPT-2 pre-trained Hugging Face model and evaluate its performance prior to fine-tuning

The following steps are taken:
- Load the GPT-2 model and tokenizer.
- Add a padding token to the tokenizer for compatibility.
- Preprocess the IMDb dataset for sequence classification.
- Evaluate the baseline performance using accuracy and F1-score.

In [74]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [75]:
# Load the tokenizer and model
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1})

# Add padding token to the tokenizer if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Load the IMDb dataset
dataset = load_dataset("imdb")
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)) 
test_dataset = dataset["test"].shuffle(seed=42).select(range(500))


# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10,
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate the model
baseline_metrics = trainer.evaluate()
print("Baseline Performance:", baseline_metrics)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Baseline Performance: {'eval_loss': 0.8299095630645752, 'eval_model_preparation_time': 0.0007, 'eval_accuracy': 0.51, 'eval_f1': 0.35017791538992193, 'eval_runtime': 62.6302, 'eval_samples_per_second': 7.983, 'eval_steps_per_second': 7.983}


#### Key Observation ####
The metrics suggest that while the model has decent initialization and speed, its performance (accuracy and F1 score) leaves significant room for improvement

## Performing Parameter-Efficient Fine-Tuning

create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In this section:
- A LoRA configuration is created with specified parameters for adaptation.
- The PEFT model is trained and fine-tuned parameters are saved for later use.

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    task_type="SEQ_CLS", ## SEQ_CLS for sequence classification tasks
    inference_mode=False, 
    lora_dropout=0.2
)

# Create PEFT model
peft_model = get_peft_model(model, lora_config)

# Update the Trainer to use the PEFT model
trainer.model = peft_model

# Train the model
trainer.train()

# Save the PEFT model weights
peft_model.save_pretrained("gpt-lora")

/opt/anaconda3/lib/python3.12/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1
1,1.241500,0.918445,0.002200,0.830000,0.829076
2,1.399800,0.845975,0.002200,0.876000,0.875672
3,0.000100,0.816857,0.002200,0.880000,0.879731


In [77]:
# check trainable parameters of model
peft_model.print_trainable_parameters()

trainable params: 296,448 || all params: 124,737,792 || trainable%: 0.2377


In [78]:
peft_metrics = trainer.evaluate()
print("\nFinal Results:")
print("Base model metrics:", baseline_metrics)
print("PEFT model metrics:", peft_metrics)


Final Results:
Base model metrics: {'eval_loss': 0.8299095630645752, 'eval_model_preparation_time': 0.0007, 'eval_accuracy': 0.51, 'eval_f1': 0.35017791538992193, 'eval_runtime': 62.6302, 'eval_samples_per_second': 7.983, 'eval_steps_per_second': 7.983}
PEFT model metrics: {'eval_loss': 0.8168573975563049, 'eval_model_preparation_time': 0.0022, 'eval_accuracy': 0.88, 'eval_f1': 0.8797307692307693, 'eval_runtime': 66.1406, 'eval_samples_per_second': 7.56, 'eval_steps_per_second': 7.56, 'epoch': 3.0}


#### Key Observations ####
- The PEFT model significantly outperforms the Base Model in accuracy and F1 score, indicating better generalization and effectiveness.
- The PEFT model slightly reduces the evaluation loss compared to the Base Model.
- The PEFT model requires slightly more preparation time, which is negligible compared to the performance gain.

## Performing Inference with a PEFT Model

Load the saved PEFT model weights and evaluate the performance of the trained PEFT model. The results to the results from prior to fine-tuning.

In [79]:
from peft import PeftModelForSequenceClassification

# Reload the PEFT model
fine_tuned_model = PeftModelForSequenceClassification.from_pretrained(model,"gpt-lora")

# Tokenize sample texts
sample_texts = [
    "The movie was absolutely wonderful! A masterpiece.",
    "Terrible movie. I would not recommend it to anyone.",
    "It was just okay, nothing too special.",
]
inputs = tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True)

# Move model and inputs to device
device = "cuda" if torch.cuda.is_available() else "cpu"
fine_tuned_model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Perform inference
outputs = fine_tuned_model(**inputs)
predictions = np.argmax(outputs.logits.detach().numpy(), axis=-1)
print("Predictions:", predictions)


Predictions: [1 0 0]


### Key Observations ###

- The model accurately classified one review as positive (1), which was a highly favorable comment about the movie.
- It correctly identified a negative review as negative (0), showcasing its ability to discern critical feedback.
- A neutral or mixed review was also classified as negative (0), which might indicate a lack of nuance in distinguishing between neutral and negative sentiments.

Overall, the model performed well in identifying clear positive and negative sentiments but might need refinement to handle more nuanced or neutral statements effectively.